# بخش دوم از جمع آوری دادها
در این بخش اطلاعات مربوط به تاریخچه پرتاب ها ارائه شده در سایت ویکی پدیا مورد استخراج قرار می گیرد


### لینک صفحه مربوطه

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


In [ ]:
# نصب کتابخانه های مورد نیاز
#%conda install beautifulsoup4
#%conda install requests

In [2]:
# فراخوانی کتابخانه های مورد نیاز
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
#آدرس مورد نظر
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# دریافت اطلاعات از آدرس و اختصاص آن به یک متغییر
response = requests.get(static_url)

# متن پاسخ را تفکیک می کنیم BeautifulSoup() با استفاده از 
soup = BeautifulSoup(response.content)

#برای چک کردن عملکرد درست مراحل فوق
#soup.title

In [ ]:
# تمامی جدول ها را جدا می کنیم و به یک متغییر جدید اختصاص می دهیم
html_tables = soup.find_all('table')

#بررسی مشخصات جدول سوم و مورد نظر
first_launch_table = html_tables[2]
print(first_launch_table)

In [ ]:
# استخراج عنوان ستون های جدول 
column_names = []

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # فیلتر کردن اعداد و جاهای خالی
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name
        

for row in first_launch_table.find_all('th'):
    column = extract_column_from_header(row)
    column_names.append(column)

#print(column_names)

In [ ]:
# ایجاد یک دیکشنری با استفاده از عنوان های استخراج شده در مرحله قبل

launch_dict= dict.fromkeys(column_names)

# حدف عنوان نامربوط از دیکشنری
del launch_dict['Date and time ( )']

# را برابر با یک لیست خالی قرار می دهیم value
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# چند مقدار جدید به دکشنری اضافه می کنیم
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [ ]:
# توابع کمکی برای استخراج مقادیر
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

In [ ]:
extracted_row = 0
#استخراج جدول مورد نظر 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # گرفتن سطرها 
    for rows in table.find_all("tr"):
        #بررسی می کنیم آیا عنوان جدول متناسب با شماره پرتاب متناظر هست 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #عناصر جدول را استخراج می کنیم
        row=rows.find_all('td')
        #اگر یک عدد است آنرا درون دیکشنری قرار بده 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append('bv2')
            #print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            
            # Customer
            customer = row[6].a.string
            launch_dict['Customer'].append(customer)
            #print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)

In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
#df.head()

In [ ]:
df.to_csv('spacex_web_scraped.csv', index=False)